# Advent of code 2022 - 08

In [1]:
from utils import read_txt_file

## Code

In [68]:
l = [1, 2, 3]
l[::-1]

[3, 2, 1]

In [76]:
class TreeGrid:
    def __init__(self, input_txt: str, debug: bool = False):
        self.debug = debug
        self.input_txt = input_txt
        # if debug:
        #     print(input_txt)

        rows = [x for x in self.input_txt.split("\n") if x != ""]
        self.height_grid = []
        for row in rows:
            self.height_grid.append([int(x) for x in list(row)])

        self.n = len(self.height_grid)
        self.m = len(self.height_grid[0])

        self.visibility_grid = []
        self.scenic_score_grid = []

    def print_colored_grid(self):
        output = ""
        for i in range(self.n):
            for j in range(self.m):
                if self.visibility_grid[i][j]:
                    output += str(self.height_grid[i][j])
                else:
                    output += f"\x1b[31m{self.height_grid[i][j]}\x1b[0m"
                    pass
            output += "\n"
        print(output)

    def _get_max_height_of_neighbors(self, which: str, i: int, j: int):
        if which == "left":
            trees_to_left = [self.height_grid[i][x] for x in range(j)]
            # print(f"i: {i}, j: {j}, height: {self.height_grid[i][j]} -> {trees_to_left}")
            return max(trees_to_left + [-1])  # avoid error with "max([])"
        elif which == "right":
            trees_to_right = [self.height_grid[i][x] for x in range(j + 1, self.m)]
            return max(trees_to_right + [-1])  # avoid error with "max([])"
        elif which == "top":
            trees_to_top = [self.height_grid[x][j] for x in range(i)]
            return max(trees_to_top + [-1])  # avoid error with "max([])"
        elif which == "bottom":
            trees_to_bottom = [self.height_grid[x][j] for x in range(i + 1, self.n)]
            return max(trees_to_bottom + [-1])  # avoid error with "max([])"
        else:
            raise ValueError(f"Wrong value for which: '{which}'")

    def check_visibility(self, i, j):
        height_current_tree = self.height_grid[i][j]

        max_heights_neighbors = [
            self._get_max_height_of_neighbors(which="left", i=i, j=j),
            self._get_max_height_of_neighbors(which="right", i=i, j=j),
            self._get_max_height_of_neighbors(which="top", i=i, j=j),
            self._get_max_height_of_neighbors(which="bottom", i=i, j=j),
        ]
        return any([height_current_tree > x for x in max_heights_neighbors])

    def compute_visibility(self):
        for i in range(self.n):
            self.visibility_grid.append([])
            for j in range(self.m):
                self.visibility_grid[i].append([])
                self.visibility_grid[i][j] = self.check_visibility(i, j)

    def _compute_view_distance(self, me: int, neighbors: list[int]):
        if len(neighbors) == 0:
            return 0

        view_dist = 0
        for i in neighbors:
            view_dist += 1
            if i >= me:
                return view_dist
        return view_dist  # no tree blocking until the end of the forest

    def _get_local_scenic_score(self, which: str, i: int, j: int):
        height_current_tree = self.height_grid[i][j]
        if which == "left":
            trees_to_left = [self.height_grid[i][x] for x in range(j)]
            trees_to_left = trees_to_left[::-1]
            view = self._compute_view_distance(height_current_tree, trees_to_left)
            # print(f"i: {i}, j: {j}, height: {self.height_grid[i][j]} -> {trees_to_left} -> {view}")
            return view

        elif which == "right":
            trees_to_right = [self.height_grid[i][x] for x in range(j + 1, self.m)]
            view = self._compute_view_distance(height_current_tree, trees_to_right)
            return view

        elif which == "top":
            trees_to_top = [self.height_grid[x][j] for x in range(i)]
            trees_to_top = trees_to_top[::-1]
            view = self._compute_view_distance(height_current_tree, trees_to_top)
            return view

        elif which == "bottom":
            trees_to_bottom = [self.height_grid[x][j] for x in range(i + 1, self.n)]
            view = self._compute_view_distance(height_current_tree, trees_to_bottom)
            return view

        else:
            raise ValueError(f"Wrong value for which: '{which}'")

    def compute_scenic_score(self, i, j):

        return (
            self._get_local_scenic_score(which="left", i=i, j=j)
            * self._get_local_scenic_score(which="right", i=i, j=j)
            * self._get_local_scenic_score(which="top", i=i, j=j)
            * self._get_local_scenic_score(which="bottom", i=i, j=j)
        )

    def create_scenic_score_grid(self):
        for i in range(self.n):
            self.scenic_score_grid.append([])
            for j in range(self.m):
                self.scenic_score_grid[i].append([])
                self.scenic_score_grid[i][j] = self.compute_scenic_score(i, j)


def solve_08(path: str, is_part_2: bool = False, debug: bool = False):
    input_txt = read_txt_file(path)

    grid = TreeGrid(input_txt=input_txt, debug=debug)

    if not is_part_2:
        grid.compute_visibility()
        if debug:
            grid.print_colored_grid()
        return sum([sum(x) for x in grid.visibility_grid])
    else:
        grid.create_scenic_score_grid()
        # print(grid.scenic_score_grid)
        return max([max(x) for x in grid.scenic_score_grid])


# Run some tests
example_file = f"inputs/08_example.txt"
assert solve_08(example_file, debug=False) == 21
assert solve_08(example_file, is_part_2=True, debug=False) == 8

input_file = "inputs/08_input.txt"
assert solve_08(input_file, debug=False) == 1807

## Example

In [50]:
example_file = f"inputs/08_example.txt"

In [59]:
solve_08(example_file, debug=True)

30373
25512
65332
33549
35390



21

In [77]:
solve_08(example_file, is_part_2=True, debug=False)

8

## Puzzle

In [53]:
input_file = "inputs/08_input.txt"

In [60]:
solve_08(input_file, debug=True)

200120010031113332213034140102430141241124321111512323442304040044220141342121012210022220211111200
111021120230012003124233312003010303204555454332453445353125225214221214442214301113012111020220220
010000112330232223210123331341111021455213441355415452453433542411022110033014112420103200010200112
210220131023203033023421012411405531115123145115445215311422135252541100222231431331003220231121021
020112210133112114444340100410411231114131452544155151415233415312511222041221113442132231003203100
121210122332301142321211440441113455552434332511455325315545354524142541534411112013143013120333022
002022223332143323320030004111431434153352153321414154453442552321332341355132444030442110103203212
212130023213100232130230411132244242133251435525323326611211534441513532233340420321144212221332012
112201210213131241203033341411115513233426453426433653356253533133113511115222441140433214102332200
212012000101044144022321521321444445566546646625625665232323663631331412343532242340023204432003233


1807

In [78]:
solve_08(input_file, is_part_2=True, debug=False)

480000